In [2]:
# this notebook iterate over neuron 4D files (already masked and filtered) 
# and stream-pipeline the defined analysis steps 
# and save the output into one dataframe per neuron with the following columns
# Output_name : XXXXXXXXX define
# output_cols: XXXXXX define

In [1]:
data_path = '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/'
save_path = '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/output/'

##
stab_limit = 5 #(No. of timepoints for stable branches calculation)
start_age = 37
subtype='A' #define the subtype to be used in DGI func
analysis_type = '3D' #define whether the analysis_type is 2D or 3D

# entry point (t,z,y,x)
# entry_point = [(0,16,240,240), (0,33,185,250),  for subtype A
#                 (0,28,195,270), (0,44,225,170), 
#                 (0,43,280,250), (0,24,210,260),  
#                 (0,31,235,240), (0,27,290,245)]
entry_points = {'211104_P36_N1': (0,25,220,230),
                '211104_P36_N3': (0,27,110,250),
                '211107_P36_N2': (0,27,225,235),
                '211107_P36_N3': (0,27,225,235),
                '211113_P36_N1': (0,26,200,220),
                '211114_P36_N3': (0,28,260,200),
                '211121_P36.5_N3': (0,34,195,165),
                '220127_P36_N1': (0,19,210,220),
                '220209_P36_N1': (0,27,225,235),
                '211011_P37_N1': (0,38,150,200),
                '211104_P36_N2': (0,16,240,240), 
                '211107_P36_N1': (0,33,185,250),
                '211113_P36_N2': (0,28,195,270), 
                '211114_P36_N1': (0,44,225,170), 
                '211114_P36_N2': (0,43,280,250), 
                '220209_P36_N2': (0,27,290,245),  
                '211121_P36.5_N1': (0,24,210,260), 
                '211121_P36.5_N2': (0,31,235,240),
                '211028_P36_N1': (0,25,210,240)}


if save_path[-1] != '/':
    save_path += '/'
if data_path[-1] != '/':
    data_path += '/'

In [2]:
# if the analysis is done on 2D max_projection. Then, remove Z from entry-points
if analysis_type == '2D':
    for neu, ent in entry_points.items():
        entry_points[neu] = (ent[0],ent[-2],ent[-1])
entry_points

{'211104_P36_N1': (0, 25, 220, 230),
 '211104_P36_N3': (0, 27, 110, 250),
 '211107_P36_N2': (0, 27, 225, 235),
 '211107_P36_N3': (0, 27, 225, 235),
 '211113_P36_N1': (0, 26, 200, 220),
 '211114_P36_N3': (0, 28, 260, 200),
 '211121_P36.5_N3': (0, 34, 195, 165),
 '220127_P36_N1': (0, 19, 210, 220),
 '220209_P36_N1': (0, 27, 225, 235),
 '211011_P37_N1': (0, 38, 150, 200),
 '211104_P36_N2': (0, 16, 240, 240),
 '211107_P36_N1': (0, 33, 185, 250),
 '211113_P36_N2': (0, 28, 195, 270),
 '211114_P36_N1': (0, 44, 225, 170),
 '211114_P36_N2': (0, 43, 280, 250),
 '220209_P36_N2': (0, 27, 290, 245),
 '211121_P36.5_N1': (0, 24, 210, 260),
 '211121_P36.5_N2': (0, 31, 235, 240),
 '211028_P36_N1': (0, 25, 210, 240)}

In [3]:
import os
import utils.analysis as analysis
import utils.datautils as datautils
import matplotlib.pyplot as plt
import tifffile as tif
import numpy as np
import re
from tqdm import tqdm
import pandas as pd
import math
from skimage.measure import regionprops
from read_roi import read_roi_zip as col_zip
from scipy.spatial import ConvexHull, convex_hull_plot_2d

In [4]:
N_folders = [name for name in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, name))]
N_folders = [f for f in N_folders if '_P' in f]
N_folders = sorted(N_folders)
N_folders

['211104_P36_N2',
 '211107_P36_N1',
 '211113_P36_N2',
 '211114_P36_N1',
 '211114_P36_N2',
 '211121_P36.5_N1',
 '211121_P36.5_N2',
 '220209_P36_N2']

In [5]:
for i, fold in tqdm(enumerate(N_folders)):
    try:
        entry_point = entry_points[fold]
        circ = lambda r: (4 * math.pi * r.area) / (r.perimeter * r.perimeter)
        for i in [1]:
            start_im = re.search('P(.*)_N', fold)
            start_im = float(start_im.group(1))
            start_ana = int((start_age - start_im)/0.25)
            files_list = datautils.get_file_names(data_path+fold, group_by='', 
                                                order=True, nested_files=False, 
                                                criteria='')
            neu_path = [f for f in files_list if 'clahe' in f.lower()][0]
            neuron = tif.imread(neu_path)
            mask_path = [f for f in files_list if 'mask' in f.lower()][0]
            mask = tif.imread(mask_path)
            neuron[mask==0] = 0
            neuron = neuron[start_ana:]
            neuron[neuron != 0] = 1
            lifetimes = analysis.cal_lifetimes(neuron, save=False, 
                                            save_path=save_path, save_file='', 
                                            xy_pixel=0.076, z_pixel=0.4)
            stable_neuron = analysis.stable_N(lifetimes, stab_limit=stab_limit, save=True, 
                                                save_path=save_path, save_file=fold, 
                                                xy_pixel=0.076, z_pixel=0.4) 
            neu_2D = neuron.max(axis=1)
            stable_2D = stable_neuron.max(axis=1)
            if analysis_type=='3D':
                image = neuron
                stable_im = stable_neuron
            elif analysis_type=='2D':
                image = neu_2D
                stable_im = stable_2D
            else:
                print('the analysis type is not recognized')
                break
            output_results = analysis.N_volume(image, stable_im, normalize=False, 
                                                start_t=start_age, plot=False, 
                                                save=False, save_path=save_path, 
                                                save_file='Vol_'+fold)
            output_results['growthR_all'] = output_results.vol_all.diff()/output_results.vol_all
            output_results['growthR_stable'] = output_results.vol_stable.diff()/output_results.vol_stable
            DGIs = analysis.calculate_DGI(entry_point, stable_im, 
                                subtype=subtype,
                                start_t=start_age, 
                                save=True, save_path='', 
                                save_file='')
            output_results = pd.merge(output_results, DGIs, on='timepoints')
            cir_im = stable_2D.copy()
            # cir_im[stable_2D!=0] = 1
            # circularity = []
            # circularity2 = []
            # for st in cir_im:
            #     circularity.append([circ(region) for region in regionprops(st)])
            #     img_PC = np.argwhere(st)
            #     try:
            #         for i in [1]:
            #             hull = ConvexHull(img_PC, qhull_options='Qc')
            #             circ2 = 4*np.pi *hull.volume/(hull.area*hull.area)
            #             circularity2.append(circ2)
            #     except:
            #         circularity2.append(0)
            # circularity = [s[0] for s in circularity[4:]]
            # circularity2 = circularity2[4:]
            # output_results['circularity'] = circularity
            # output_results['circularity2'] = circularity2
            area, perimeter, density, circularity = analysis.cal_circularity(cir_im)
            circularity = circularity[4:]
            density = density[4:]
            area = area[4:]
            perimeter = perimeter[4:]
            output_results['circularity'] = circularity
            output_results['density'] = density
            output_results['area'] = area
            output_results['perimeter'] = perimeter
            output_file = save_path+fold+'_s'+subtype+'_output.csv'
            output_file = output_file.replace('.tif','')
            output_results.to_csv(output_file, sep=';')
            # cols_path = [f for f in files_list if 'colrois' in f.lower()][0]
            # Cols = col_zip(cols_path)
            # analysis.col_occupancy(stable_2D, cols_zip=Cols, norm_cols=Cols, 
            #                             normalize_cols=True, nor_fact=1, 
            #                             start_t=start_age, 
            #                             plot=True, save=True, 
            #                             save_path=save_path, 
            #                             save_file='st_2Dpx_'+fold+'_s'+subtype)
            # analysis.col_occupancy(stable_neuron, cols_zip=Cols, norm_cols=Cols, 
            #                             normalize_cols=True, nor_fact=1, 
            #                             start_t=start_age, 
            #                             plot=True, save=True, 
            #                             save_path=save_path, 
            #                             save_file='st_3Dpx_'+fold+'_s'+subtype)
            # del neuron, lifetimes, stable_neuron, neu_2D, stable_2D, output_results, DGIs
    except:
        pass

0it [00:00, ?it/s]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211104_P36_N2/211104_P36_N2_R270_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211104_P36_N2/211104_P36_N2_R270_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211104_P36_N2/211104_P36_N2_R270_ColROIs.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211104_P36_N2/211104_P36_N2_R270_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211104_P36_N2/211104_P36_N2_CropROI.roi']


100%|██████████| 85/85 [01:59<00:00,  1.40s/it]
1it [02:24, 144.72s/it]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211107_P36_N1/211107_P36_N1_Z30.70_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211107_P36_N1/211107_P36_N1_Z30.70_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211107_P36_N1/211107_P36_N1_R265_mask1.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211107_P36_N1/211107_P36_N1_R265_Z30.70_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211107_P36_N1/211107_P36_N1_R265_ColRois.zip']


100%|██████████| 88/88 [01:50<00:00,  1.26s/it]
2it [04:31, 134.21s/it]

...

100%|██████████| 90/90 [01:22<00:00,  1.09it/s]
6it [13:25, 132.77s/it]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211121_P36.5_N2/211121_P36.5_N2_Z18.62_CropRoi.roi', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211121_P36.5_N2/211121_P36.5_N2_R302_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211121_P36.5_N2/211121_P36.5_N2_R302_clahe_m.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211121_P36.5_N2/211121_P36.5_N2_R302_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211121_P36.5_N2/211121_P36.5_N2_R302_ColRois.zip']


100%|██████████| 90/90 [01:51<00:00,  1.24s/it]
7it [15:34, 131.37s/it]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/220209_P36_N2/220209_P36_N2_FlipV_R255_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/220209_P36_N2/220209_P36_N2_FlipV_R255_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/220209_P36_N2/220209_P36_N2_FlipV_R255_5D_new.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/220209_P36_N2/220209_P36_N2_CropROI.roi']


100%|██████████| 84/84 [01:53<00:00,  1.35s/it]
8it [17:42, 132.86s/it]
